In [ ]:
import pandas as pd

In [ ]:
flight_df = pd.read_csv(r"C:\Users\macie\OneDrive\Pulpit\Studia PJATK\flight_df_01.csv", delimiter=';')

In [ ]:
aircraft_df = pd.read_csv(r"C:\Users\macie\OneDrive\Pulpit\Studia PJATK\aircraft.csv")

In [ ]:
aircraft_df

In [ ]:
aircraft_df.drop(aircraft_df[['Unnamed: 0','NUMBER_OF_SEATS']], axis = 1, inplace = True)
aircraft_df.drop_duplicates(inplace = True)

In [ ]:
aircraft_df

## Modyfikacja aircraft_df 

In [ ]:
aircraft_df.sort_values(['TAIL_NUM', 'MANUFACTURE_YEAR'], ascending = [True, False], inplace = True )

## Połączenie aircraft_df oraz flight_df 

In [ ]:
print(flight_df.columns)
print(aircraft_df.columns)


In [ ]:
tmp_flight_df = flight_df.merge(aircraft_df, how = 'inner', left_on = 'TAIL_NUM', right_on = 'TAIL_NUM')

In [ ]:
tmp_flight_df.duplicated().unique()

In [ ]:
flight_df = tmp_flight_df

## Opóźnienia vs rok produkcji samolotu part 1 
Wyznaczymy zależności roku produkcji do częstotliwości opóźnień 

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
delays_by_manufacture_year_df = flight_df.loc[flight_df['IS_DELAYED'] == 1].groupby(by = 'MANUFACTURE_YEAR')['IS_DELAYED'].count()/flight_df.groupby(by = 'MANUFACTURE_YEAR')['IS_DELAYED'].count()

In [ ]:
delays_by_manufacture_year_df

Modyfikacja przyszłego wykresu aby prezentował tylko te roczniki, które odbyły więcejk niż 10000 lotów

In [ ]:
over_10k_df = flight_df.groupby(by = 'MANUFACTURE_YEAR')['IS_DELAYED'].count()
over_10k_df = pd.DataFrame(over_10k_df)
over_10k_df = over_10k_df.loc[over_10k_df['IS_DELAYED'] > 10000]

In [ ]:
over_10k_df = over_10k_df.merge(delays_by_manufacture_year_df,
                               how = 'left',
                               left_on = 'MANUFACTURE_YEAR',
                               right_on = 'MANUFACTURE_YEAR')

In [ ]:
fig = px.bar(over_10k_df, y = 'IS_DELAYED_y', title = 'proportion_delays/over 10k')
fig.show()

## Opóźnienia vs rok produkcji samolotu part 2

In [ ]:
flight_df['manufacture_year_agg'] = pd.cut(flight_df['MANUFACTURE_YEAR'], bins = [*range(int(flight_df['MANUFACTURE_YEAR'].min()), int(flight_df['MANUFACTURE_YEAR'].max()), 3)])

In [ ]:
flight_delays_by_manufacture_year_agg_df = flight_df.loc[flight_df['IS_DELAYED'] == 1].groupby(by = 'manufacture_year_agg')['IS_DELAYED'].count()
flight_delays_by_manufacture_year_agg_df = pd.DataFrame(flight_delays_by_manufacture_year_agg_df).sort_values(by = 'manufacture_year_agg')
flight_delays_by_manufacture_year_agg_df['m_y'] = flight_delays_by_manufacture_year_agg_df.index

In [ ]:
flight_delays_by_manufacture_year_agg_df['m_y'] = flight_delays_by_manufacture_year_agg_df['m_y'].astype(str)

In [ ]:
plt.figure(figsize=(15,10))
plt.xticks(rotation=90)
plt.bar(x = flight_delays_by_manufacture_year_agg_df['m_y'], height = flight_delays_by_manufacture_year_agg_df['IS_DELAYED'])

#### Wyznaczamy top 5 roczników produkcji - sortując według liczby wykonanych lotów, pamiętając o wyświetleniu również wartości opóźnienia

In [ ]:
delays_by_manufacture_year_df

In [ ]:
top_manufactured_df = over_10k_df.sort_values(by = 'IS_DELAYED_x', ascending = False).head(5)

In [ ]:
top_manufactured_df

In [ ]:
top_manufactured_df.rename(columns = {'IS_DELAYED_x': 'numberf of flights', 'IS_DELAYED_y': 'percentage of delays'},inplace = True)

In [ ]:
top_manufactured_df

In [ ]:
flight_df_02 = flight_df
flight_df_02.to_csv("flight_df_02.csv", sep = ';', encoding = 'utf-8', index = False)